In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

### Prepare input file

In [3]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    model=MODEL_HUB_ID,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput, BaseAssistantOutput


class SomeInput(BaseAssistantInput):
    pass


class SomeOutput(BaseAssistantOutput):
    result: int

In [4]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMBatchResult,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'
some_input = SomeInput()

# request = LLMRequest(
#     conversation=LLMConversation(
#         messages=[
#             LLMMessage(role='system', content='You are a helpful assistant.'),
#             LLMMessage(role='user', content='what is 2+2'),
#         ]
#     ),
#     params=LLMParams(
#         model=MODEL_HUB_ID,
#         temperature=0,
#         response_type=SomeOutput,
#         max_completion_tokens=10,
#         metadata={'input_id': str(some_input.id)},
#     ),
# )

# batch_request: LLMBatchRequest = LLMBatchRequest(requests=[request])

_requests = [
    LLMRequest(
        conversation=LLMConversation(
            messages=[
                LLMMessage(role='system', content='You are a helpful assistant.'),
                LLMMessage(role='user', content=f'what is {i}+2'),
            ]
        ),
        params=LLMParams(
            model=MODEL_HUB_ID,
            temperature=0,
            response_type=SomeOutput,
            max_completion_tokens=10,
            metadata={'input_id': str(some_input.id)},
        ),
    )
    for i in range(200)
]

batch_request: LLMBatchRequest = LLMBatchRequest(requests=_requests)

In [5]:
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]
lines = [json.dumps(request, separators=(',', ':')) for request in requests]
jsonl_str = '\n'.join(lines)
jsonl_bytes = jsonl_str.encode('utf-8')

In [7]:
request_dict = json.loads(lines[0])

In [8]:
result = await client.chat_completion(**request_dict)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1744125157, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [40]:
response_list = LLMResponseListMapping.to_source(
    result,
    request_id=request.id,
    input_id=request_dict['extra_body']['input_id'],
    response_type=SomeOutput,
)
response_list

LLMResponseList(id=UUID('f7fd3b29-2772-468d-831f-b160bf1dfaa0'), request_id=UUID('cb02d5a3-bed8-4109-b06c-04d585081ef3'), responses=[LLMResponse(id=UUID('3976268b-adbf-494a-b88b-fda24781d477'), content=BoundAssistantOutput(id=UUID('bd7fdedd-f32a-4d7f-845d-0a21413828b6'), input_id=UUID('c882baa3-2047-4f65-b8df-bd176bba3b2c'), result=4), logprobs=None)])

In [ ]:
input_file_path = Path(f'.tmp/input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

### Apptainer

In [5]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

tgi_batch_llm = infrastructure_container.tgi_batch_llm()

In [6]:
await tgi_batch_llm.init_resources()

2025-04-26 19:45:43,639 - INFO - Host canonicalization disabled
2025-04-26 19:45:43,639 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-04-26 19:45:43,683 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-26 19:45:43,684 - INFO - [conn=0]   Local address: 172.22.0.10, port 42580
2025-04-26 19:45:43,684 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-04-26 19:45:43,731 - INFO - [conn=0] Beginning auth for user lpanic
2025-04-26 19:45:44,004 - INFO - [conn=0] Auth for user lpanic succeeded
2025-04-26 19:45:44,008 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-04-26 19:45:44,135 - INFO - [conn=0, chan=0]   Command: mkdir -p tgi_default_root
2025-04-26 19:45:44,293 - INFO - [conn=0, chan=0] Received exit status 0
2025-04-26 19:45:44,295 - INFO - [conn=0, chan=0] Received channel close
2025-04-26 19:45:44,297 - INFO - [conn=0, chan=0] Channel closed
2025-04-26 19:45:44,298 - INFO - Command `mkdir -p tgi_default_

In [ ]:
res = await tgi_batch_llm.batch_generate(
    batch_request=batch_request,
    response_type=SomeOutput,
    poll_interval=3 * 60,
    max_tokens_per_day=None,
    max_num_retries=0,
)
res

2025-04-26 19:50:52,883 - INFO - Host canonicalization disabled
2025-04-26 19:50:52,884 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-04-26 19:50:52,892 - INFO - [conn=25] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-26 19:50:52,892 - INFO - [conn=25]   Local address: 172.22.0.10, port 59458
2025-04-26 19:50:52,892 - INFO - [conn=25]   Peer address: 161.53.2.37, port 22
2025-04-26 19:50:52,924 - INFO - [conn=25] Beginning auth for user lpanic
2025-04-26 19:50:53,193 - INFO - [conn=25] Auth for user lpanic succeeded
2025-04-26 19:50:53,196 - INFO - [conn=25, chan=0] Requesting new SSH session
2025-04-26 19:50:53,351 - INFO - [conn=25, chan=0]   Subsystem: sftp
2025-04-26 19:50:53,375 - INFO - [conn=25, chan=0] Starting SFTP client
2025-04-26 19:50:53,745 - INFO - [conn=25, chan=0] SFTP client exited
2025-04-26 19:50:53,747 - INFO - [conn=25, chan=0] Closing channel
2025-04-26 19:50:53,749 - INFO - [conn=25, chan=0] Received exit status 0
202

In [4]:
from pathlib import Path


sftp_client = infrastructure_container.sftp_client()

# tar -czvf snapshot.tar.gz data

# TODO download dir
await sftp_client.download(
    Path('tgi_default_root/snapshot.tar.gz'), Path('../.tmp/snapshot.tar.gz')
)

# tar -xzvf snapshot.tar.gz

2025-04-26 17:50:41,360 - INFO - Host canonicalization disabled
2025-04-26 17:50:41,360 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-04-26 17:50:41,370 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-26 17:50:41,370 - INFO - [conn=0]   Local address: 172.22.0.10, port 59724
2025-04-26 17:50:41,370 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-04-26 17:50:41,446 - INFO - [conn=0] Beginning auth for user lpanic
2025-04-26 17:50:41,712 - INFO - [conn=0] Auth for user lpanic succeeded
2025-04-26 17:50:41,713 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-04-26 17:50:41,842 - INFO - [conn=0, chan=0]   Subsystem: sftp
2025-04-26 17:50:41,848 - INFO - [conn=0, chan=0] Starting SFTP client
2025-04-26 17:50:42,045 - INFO - [conn=0, chan=0] Received exit status 0
2025-04-26 17:50:42,045 - INFO - [conn=0, chan=0] Received channel close
2025-04-26 17:50:42,046 - INFO - [conn=0, chan=0] SFTP client exited
2025-04

In [ ]:
from os import environ

from decouple import config


environ['PBS_O_WORKDIR'] = '../.tmp'
environ['TGI_API_KEY'] = config('HUGGINGFACE_TOKEN')
environ['MODEL_HUB_ID'] = 'microsoft/Phi-3-mini-4k-instruct'

%run ../assets/hpc/hf/tgi/tgi_client.py